# question1:
running this on github code spaces
```bash
docker run --rm \
    --name elastic \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```
and then running this
```bash
curl localhost:9200
```

`version.build_hash` is `42f05b9372a9a4a470db3b52817899b99a76ee73`